<a href="https://colab.research.google.com/github/darkmochalover/DS_TermProject_AL2/blob/main/disicion%20tree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Dictionary

In [125]:
# Data Path
raw_data_path = "/content/drive/MyDrive/DS_TermProject/data/combined_mbti_df.csv"
# data_path = "data/preprocessed_mbti_data.csv"


audio_qualities = [
    'danceability_mean',
    'valence_mean',
    'energy_mean',
    'loudness_mean',
    'acousticness_mean',
    'instrumentalness_mean',
    'liveness_mean',
]

# 장조/단조 (Major/Minor)
all_tones = [
    'Cminor_count', 'CMajor_count', 'C#/Dbminor_count', 'C#/DbMajor_count',
    'DMajor_count', 'D#_EbMajor_count', 'Eminor_count', 'EMajor_count',
    'Fminor_count', 'FMajor_count', 'F#/Gbminor_count', 'GMajor_count',
    'G#/Abminor_count', 'G#/AbMajor_count', 'Aminor_count', 'AMajor_count',
    'A#/Bbminor_count', 'BMajor_count', 'Dminor_count', 'D#_Ebminor_count',
    'Gminor_count', 'A#/BbMajor_count', 'F#/GbMajor_count', 'Bminor_count'
]

major_tones = [
    'CMajor_count', 'C#/DbMajor_count',
    'DMajor_count', 'D#_EbMajor_count', 
    'EMajor_count',
    'FMajor_count', 
    'GMajor_count', 'G#/AbMajor_count', 
    'AMajor_count', 'BMajor_count', 'A#/BbMajor_count', 
    'F#/GbMajor_count'
]
minor_tones = [
    'Cminor_count', 'C#/Dbminor_count', 
    'Eminor_count', 
    'Fminor_count', 'F#/Gbminor_count', 
    'G#/Abminor_count',  
    'Aminor_count', 'A#/Bbminor_count', 
    'Dminor_count', 'D#_Ebminor_count',
    'Gminor_count', 
    'Bminor_count'
]


# 열 이름을 바꾸기 위해 리스트로 저장함
renamed_columns =  [
    'danceability',
    'valence',
    'energy',
    'loudness',
    'acousticness',
    'instrumentalness',
    'liveness'
]


Install WandB

In [126]:
!pip install -qU wandb

In [127]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
from imblearn.over_sampling import SMOTE

import itertools

import wandb
import random

Step 1: Import W&B and Login

In [128]:
import wandb
from wandb.keras import WandbMetricsLogger, WandbModelCheckpoint, WandbEvalCallback

In [129]:
encoder_list = ['LabelEncoder']
scaler_list = ['StandardScaler', 'MinMaxScaler', 'RobustScaler', 'No Scale']
test_size_list = [0.3, 0.2, 0.1]
do_smote = ['True', 'False']

In [130]:
config = {
        'encoder' : 'LabelEncoder',
        'scaler' : 'StandardScaler',
        'test_size' : 0.3,
        "do_smote" : True
    }

# start a new wandb run to track this script
wandb.init(
    # set the wandb project where this run will be logged
    project="MBTI_playlist",
    
    # track hyperparameters and run metadata
    config = config
)

config=wandb.config


Data Loading

In [131]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd

raw_data_path = '/content/drive/MyDrive/DS_TermProject/data/combined_mbti_df.csv'
df = pd.read_csv(raw_data_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Feature Engineering

In [132]:
# Subset only measures of centers
X = df.iloc[: , :22] # 처음 22개 열을 선택해서 저장
X = df[audio_qualities] # 오디오 품질과 해당 열이 있는 열을 선택해서 저장


categories = renamed_columns[:]
X.columns = renamed_columns


# 장조/단조의 개수의 합을 계산해서 저장 (C장조, D단조, .. 이렇게 따로 계산되는거 말고, 위에 지정된 list 이용해서 sum값 넣어줌)
X['major_count'] = df[major_tones].sum(axis=1).astype('int64')
X['minor_count'] = df[minor_tones].sum(axis=1).astype('int64')

<ipython-input-132-482ca4c8bc7c>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['major_count'] = df[major_tones].sum(axis=1).astype('int64')
<ipython-input-132-482ca4c8bc7c>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['minor_count'] = df[minor_tones].sum(axis=1).astype('int64')


## Encoding & Scaling Function

In [133]:
def Encoding(df, encoding_method):
    df = df.copy()

    if(encoding_method == 'LabelEncoder'):
        encoder = LabelEncoder()
        target = encoder.fit_transform(df[['mbti']])
        

    if(encoding_method == 'OneHotEncoder'):
        encoder = OneHotEncoder(sparse=False)
        target = encoder.fit_transform(df[['mbti']])

    return target

def Scaling(scale_method, X_train, X_test):
    if( scale_method == 'No Scale'):
        return X_train, X_test

    elif(scale_method == 'StandardScaler'):
        scaler = StandardScaler()

    elif(scale_method == 'MinMaxScaler'):
        scaler = MinMaxScaler()

    elif(scale_method == 'RobustScaler'):
        scaler = RobustScaler()


    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.fit_transform(X_test)

    return X_train_scaled, X_test_scaled


In [134]:
print("Shape of X:", X.shape)
print("Shape of y:", y.shape)


Shape of X: (4081, 9)
Shape of y: (4081,)


In [137]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, recall_score, precision_score
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import cross_val_score, KFold
import itertools

# grid search(hyperparameter)
param_grid = {
    'max_depth': [None, 5, 10, 15],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'criterion': ['gini', 'entropy']
}

# random search(hyperparameter)
param_dist = {
    'max_depth': [None, 5, 10, 15],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'criterion': ['gini', 'entropy']
}

## combinations of hyperparameters
combinations = list(itertools.product(encoder_list, scaler_list, test_size_list, [do_smote]))

for encoder, scaler, test_size, do_smote in combinations:
    # Rest of the code remains unchanged

    print(f"Encoder: {encoder}, Scaler: {scaler}, Test Size: {test_size}, Smote Phase: {do_smote}")
    wandb.config.update({
        'encoder': encoder,
        'scaler': scaler,
        'test_size': test_size,
        'do_smote': do_smote
    })

    y = Encoding(df=df, encoding_method=encoder)

    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=5, test_size=test_size)

    X_train, X_test = Scaling(scale_method=scaler, X_train=X_train, X_test=X_test)

    if do_smote == 'True':
        smote = SMOTE(sampling_strategy='auto', random_state=0)
        X_train, y_train = smote.fit_resample(X_train, y_train)

    print('Train 피처/레이블 데이터 세트:', X_train.shape, y_train.shape)

    wandb.log({'Train feature 데이터 세트': X_train.shape})
    wandb.log({'Train label 데이터 세트': y_train.shape})

    # Decision Tree Model with hyperparameter tuning
    dt_model = DecisionTreeClassifier(random_state=42)

    # Grid Search
    grid_search = GridSearchCV(estimator=dt_model, param_grid=param_grid, cv=5)
    grid_search.fit(X_train, y_train)

    best_params_grid = grid_search.best_params_
    dt_model_grid = DecisionTreeClassifier(random_state=42, **best_params_grid)
    dt_model_grid.fit(X_train, y_train)
    dt_predictions_grid = dt_model_grid.predict(X_test)
    dt_accuracy_grid = accuracy_score(y_test, dt_predictions_grid)
    dt_confusion_matrix_grid = confusion_matrix(y_test, dt_predictions_grid)
    dt_f1_score_grid = f1_score(y_test, dt_predictions_grid, average='macro')
    dt_recall_grid = recall_score(y_test, dt_predictions_grid, average='macro')
    dt_precision_grid = precision_score(y_test, dt_predictions_grid, average='macro')
    print("Decision Tree Accuracy with Grid Search:", dt_accuracy_grid)
    print("Decision Tree Confusion Matrix with Grid Search:\n", dt_confusion_matrix_grid)
    print("Decision Tree F1 Score with Grid Search:", dt_f1_score_grid)
    print("Decision Tree Recall with Grid Search:", dt_recall_grid)
    print("Decision Tree Precision with Grid Search:", dt_precision_grid)
    wandb.log({'Decision Tree Accuracy with Grid Search': dt_accuracy_grid})
    wandb.log({'Decision Tree Confusion Matrix with Grid Search': dt_confusion_matrix_grid.tolist()})
    wandb.log({'Decision Tree F1 Score with Grid Search': dt_f1_score_grid})
    wandb.log({'Decision Tree Recall with Grid Search': dt_recall_grid})
    wandb.log({'Decision Tree Precision with Grid Search': dt_precision_grid})

    # Random Search
    random_search = RandomizedSearchCV(estimator=dt_model, param_distributions=param_dist, cv=5)
    random_search.fit(X_train, y_train)

    best_params_random = random_search.best_params_
    dt_model_random = DecisionTreeClassifier(random_state=42, **best_params_random)
    dt_model_random.fit(X_train, y_train)
    dt_predictions_random = dt_model_random.predict(X_test)
    dt_accuracy_random = accuracy_score(y_test, dt_predictions_random)
    dt_confusion_matrix_random = confusion_matrix(y_test, dt_predictions_random)
    dt_f1_score_random = f1_score(y_test, dt_predictions_random, average='macro')
    dt_recall_random = recall_score(y_test, dt_predictions_random, average='macro')
    dt_precision_random = precision_score(y_test, dt_predictions_random, average='macro')
    print("Decision Tree Accuracy with Random Search:", dt_accuracy_random)
    print("Decision Tree Confusion Matrix with Random Search:\n", dt_confusion_matrix_random)
    print("Decision Tree F1 Score with Random Search:", dt_f1_score_random)
    print("Decision Tree Recall with Random Search:", dt_recall_random)
    print("Decision Tree Precision with Random Search:", dt_precision_random)
    wandb.log({'Decision Tree Accuracy with Random Search': dt_accuracy_random})
    wandb.log({'Decision Tree Confusion Matrix with Random Search': dt_confusion_matrix_random.tolist()})
    wandb.log({'Decision Tree F1 Score with Random Search': dt_f1_score_random})
    wandb.log({'Decision Tree Recall with Random Search': dt_recall_random})
    wandb.log({'Decision Tree Precision with Random Search': dt_precision_random})

    # K-Fold 
    kf = KFold(n_splits=5, shuffle=True, random_state=42)

    dt_model_cv = DecisionTreeClassifier(random_state=42, **best_params_grid)
    cv_scores = cross_val_score(dt_model_cv, X_train, y_train, cv=kf, scoring='accuracy')
    cv_accuracy = np.mean(cv_scores)
    print("Decision Tree Accuracy with K-Fold Cross Validation:", cv_accuracy)
    wandb.log({'Decision Tree Accuracy with K-Fold Cross Validation': cv_accuracy})


Encoder: LabelEncoder, Scaler: StandardScaler, Test Size: 0.3, Smote Phase: ['True', 'False']


Error: ignored

In [136]:
wandb.finish()

Decision Tree Accuracy with Grid Search,▆▆█▅▃▁█▆██▅█
Decision Tree Accuracy with K-Fold Cross Validation,▁▄█▁▄█▁▄█▁▄█
Decision Tree Accuracy with Random Search,▆▇█▅▃▁█▆▃█▄█
Decision Tree F1 Score with Grid Search,▇▇█▆▅▁█▇▇█▆▇
Decision Tree F1 Score with Random Search,▇█▇▆▅▁█▇▃█▇▇
Decision Tree Precision with Grid Search,▅█▆▆▅▁▆▇▅▆▇▆
Decision Tree Precision with Random Search,▆█▆▇▅▁▆█▅▇▇▆
Decision Tree Recall with Grid Search,▇▆█▆▄▁█▆██▅█
Decision Tree Recall with Random Search,▇▇█▆▄▁█▆▅█▄█
Decision Tree Accuracy with Grid Search,0.20782
Decision Tree Accuracy with K-Fold Cross Validation,0.20234
